In [205]:
from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime
import time

#basic externals
import pandas as pd
import numpy as np
import os
import matplotlib as plt 

%matplotlib inline
%reload_ext autoreload
%autoreload 2


InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 20
pd.options.display.max_columns = 50 
pd.set_option(
    "display.float_format", lambda x: "%.3f" % x
    )  # display numbers as decimals instead of scientific value

plt.rcParams["font.size"] = (18)

In [206]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")
data_store = pd.read_csv("store.csv")

C:\Users\Maly-Fenix\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [207]:
df = pd.concat([data_train,data_test], sort=False)

## Data changes

In [208]:
df.StateHoliday = df.StateHoliday.replace(0,'no')
df.StateHoliday = df.StateHoliday.replace('0','no')
df['IsStateHoliday'] = (df.StateHoliday != 'no')

df["Date"] = pd.to_datetime(df.Date)
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day
df.shape

(1058297, 14)

In [209]:
print('Max delta between available data and wanted prediction of :')
pd.to_datetime(data_test['Date'].max()) - pd.to_datetime(data_train['Date'].max())

Max delta between available data and wanted prediction of :


Timedelta('48 days 00:00:00')

In [210]:
df["Timestamp"] = pd.DatetimeIndex(df["Date"] ).asi8
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[["Timestamp"]]=scaler.fit_transform(df[["Timestamp"]])

df = df.sort_values(["Date", 'Store'])

C:\Users\Maly-Fenix\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [211]:
df['Open'] = df['Open'].fillna(1)

In [212]:
df.describe(include='all')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Id,IsStateHoliday,Year,Month,Day,Timestamp
count,1058297.000,1058297.000,1058297,1017209.000,1017209.000,1058297.000,1058297.000,1058297,1058297.000,41088.000,1058297,1058297.000,1058297.000,1058297.000,1058297.000
unique,nan,nan,990,nan,nan,nan,nan,4,nan,nan,2,nan,nan,nan,nan
top,nan,nan,2015-05-21 00:00:00,nan,nan,nan,nan,no,nan,nan,False,nan,nan,nan,nan
freq,nan,nan,1115,nan,nan,nan,nan,1027067,nan,nan,1027067,nan,nan,nan,nan
first,nan,nan,2013-01-01 00:00:00,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,nan,nan
last,nan,nan,2015-09-17 00:00:00,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,nan,nan
mean,558.331,3.998,NaN,5773.819,633.146,0.831,0.382,NaN,0.189,20544.500,NaN,2013.878,5.944,15.618,0.490
std,321.846,1.998,NaN,3849.926,464.412,0.375,0.486,NaN,0.391,11861.228,NaN,0.795,3.298,8.785,0.289
min,1.000,1.000,NaN,0.000,0.000,0.000,0.000,NaN,0.000,1.000,NaN,2013.000,1.000,1.000,0.000
25%,280.000,2.000,NaN,3727.000,405.000,1.000,0.000,NaN,0.000,10272.750,NaN,2013.000,3.000,8.000,0.240


In [213]:
df.sample(3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Id,IsStateHoliday,Year,Month,Day,Timestamp
206321,47,2,2015-01-27,8503.000,1051.000,1.000,1,no,0,nan,False,2015,1,27,0.764
874810,321,3,2013-05-08,5625.000,578.000,1.000,0,no,0,nan,False,2013,5,8,0.128
85877,23,5,2015-05-15,6291.000,539.000,1.000,0,no,0,nan,False,2015,5,15,0.874


In [214]:
data_store.describe(include='all')

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1115.000,1115,1115,1112.000,761.000,761.000,1115.000,571.000,571.000,571
unique,nan,4,3,nan,nan,nan,nan,nan,nan,3
top,nan,a,a,nan,nan,nan,nan,nan,nan,"Jan,Apr,Jul,Oct"
freq,nan,602,593,nan,nan,nan,nan,nan,nan,335
mean,558.000,NaN,NaN,5404.901,7.225,2008.669,0.512,23.595,2011.764,NaN
std,322.017,NaN,NaN,7663.175,3.212,6.196,0.500,14.142,1.675,NaN
min,1.000,NaN,NaN,20.000,1.000,1900.000,0.000,1.000,2009.000,NaN
25%,279.500,NaN,NaN,717.500,4.000,2006.000,0.000,13.000,2011.000,NaN
50%,558.000,NaN,NaN,2325.000,8.000,2010.000,1.000,22.000,2012.000,NaN
75%,836.500,NaN,NaN,6882.500,10.000,2013.000,1.000,37.000,2013.000,NaN


#### Competition

In [215]:
data_store['CompetitionOpenSinceYear'] = data_store['CompetitionOpenSinceYear'].fillna(2050).clip(lower=2000)
data_store['CompetitionOpenSinceMonth'] = data_store['CompetitionOpenSinceMonth'].fillna(1)
data_store["CompetitionOpenSince"] = pd.to_datetime(dict(year=data_store.CompetitionOpenSinceYear , month=data_store.CompetitionOpenSinceMonth, day=1))

data_store.CompetitionDistance = data_store.CompetitionDistance.fillna(data_store.CompetitionDistance.mean())

#### Promo2

### Merging 

In [216]:
full_df = pd.merge(df, 
                     data_store, 
                     on = 'Store', how = 'left')

In [217]:
full_df = full_df.sort_values(['Date', 'Store'])

## Competition & promo since

In [218]:
full_df["CompetitionDaysOpen"] = full_df.Date.subtract(full_df.CompetitionOpenSince).dt.days
full_df.loc[full_df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
#full_df["CompetitionDaysOpenSqrt"] = np.sqrt(full_df.CompetitionDaysOpen)
full_df["CompetitionIsOpen"] = np.where(full_df.CompetitionDaysOpen>=1, 1, 0)

In [219]:
full_df["Promo2SinceYear"] = full_df["Promo2SinceYear"].fillna(2050)
full_df["Promo2SinceWeek"] = full_df["Promo2SinceWeek"].fillna(1)
full_df["Promo2Since"] = pd.to_datetime(full_df.apply(lambda x: str(int(x.Promo2SinceYear))+str(int(x.Promo2SinceWeek)) +'monday', axis=1), format='%Y%W%A')
full_df["Promo2Days"] = full_df.Date.subtract(full_df["Promo2Since"]).dt.days

full_df.loc[full_df.Promo2Days<0, "Promo2Days"] = 0
#full_df["Promo2DaysSqrt"] = np.sqrt(full_df.Promo2Days)
full_df["Promo2IsActive"] = np.where(full_df.Promo2Days>=1, 1, 0)

In [220]:
drop_cols = ["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear"
             , "Promo2SinceYear", 'Promo2SinceWeek', 'Promo2Since']
full_df = full_df.drop(drop_cols, axis = 1)

In [221]:
display_cols= full_df.columns

In [222]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(full_df.Store.values,full_df[fld].values, full_df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    full_df[pre+fld] = res

In [223]:
columns = ['SchoolHoliday', 'IsStateHoliday', 'Promo']

for col in columns:
    full_df = full_df.sort_values(['Store', 'Date'])
    get_elapsed(col, 'After')
    full_df = full_df.sort_values(['Store', 'Date'], ascending=[True, False])
    get_elapsed(col, 'Before')

In [224]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        full_df[a] = full_df[a].fillna(0).astype(int)

### Rolling sum

In [225]:
full_df = full_df.sort_values(['Date', 'Store'])

In [226]:
full_df['rolling_sales_half_year_med'] = full_df.groupby('Store')['Sales'].shift(48).rolling(180, min_periods = 0).median()
full_df['rolling_sales_prev_year_med'] = full_df.groupby('Store')['Sales'].shift(361).rolling(8, min_periods = 0).median()
full_df['rolling_sales_prev_year2_med'] = full_df.groupby('Store')['Sales'].shift(355).rolling(16, min_periods = 0).median()
full_df['rolling_sales_past_quarter_med'] = full_df.groupby('Store')['Sales'].shift(48).rolling(30, min_periods = 0).median()

#full_df['rolling_sales_half_year_mean'] = full_df.groupby('Store')['Sales'].shift(48).rolling(180, min_periods = 0).mean()
#full_df['rolling_sales_prev_year_mean'] = full_df.groupby('Store')['Sales'].shift(361).rolling(8, min_periods = 0).mean()

#full_df['rolling_customers_half_year_med'] = full_df.groupby('Store')['Customers'].shift(48).rolling(180, min_periods = 0).median()
#full_df['rolling_customers_prev_year_med'] = full_df.groupby('Store')['Customers'].shift(361).rolling(8, min_periods = 0).median()

#full_df['rolling_customers_half_year_mean'] = full_df.groupby('Store')['Customers'].shift(48).rolling(180, min_periods = 0).mean()
#full_df['rolling_customers_prev_year_med'] = full_df.groupby('Store')['Customers'].shift(361).rolling(8, min_periods = 0).mean()

In [227]:
#full_df.loc[full_df['rolling_sales_half_year_med'].isna(), 'rolling_sales_half_year_med'] = full_df[full_df['rolling_sales_half_year_med'].isna()].groupby('Store')['Sales'].shift(0).rolling(180, min_periods = 0).median()
#full_df.loc[full_df['rolling_sales_prev_year_med'].isna(), 'rolling_sales_prev_year_med'] = full_df[full_df['rolling_sales_prev_year_med'].isna()].groupby('Store')['Sales'].shift(0).rolling(180, min_periods = 0).median()

#full_df.loc[full_df['rolling_sales_half_year_mean'].isna(), 'rolling_sales_half_year_mean'] = full_df[full_df['rolling_sales_half_year_mean'].isna()].groupby('Store')['Sales'].shift(0).rolling(180, min_periods = 0).mean()
#full_df.loc[full_df['rolling_sales_prev_year_mean'].isna(), 'rolling_sales_prev_year_mean'] = 1 #full_df[full_df['rolling_sales_prev_year_mean'].isna()].groupby('Store')['Sales'].shift(0).rolling(180, min_periods = 0).mean()

#full_df.loc[full_df['rolling_customers_half_year_med'].isna(), 'rolling_customers_half_year_med'] = full_df[full_df['rolling_customers_half_year_med'].isna()].groupby('Store')['Customers'].shift(0).rolling(180, min_periods = 0).median()
#full_df.loc[full_df['rolling_customers_prev_year_med'].isna(), 'rolling_customers_prev_year_med'] = full_df[full_df['rolling_customers_prev_year_med'].isna()].groupby('Store')['Customers'].shift(0).rolling(180, min_periods = 0).median()

#full_df.loc[full_df['rolling_customers_half_year_mean'].isna(), 'rolling_customers_half_year_mean'] = full_df[full_df['rolling_customers_half_year_mean'].isna()].groupby('Store')['Customers'].shift(0).rolling(180, min_periods = 0).mean()
#full_df.loc[full_df['rolling_customers_prev_year_mean'].isna(), 'rolling_customers_prev_year_mean'] = 1 #full_df[full_df['rolling_customers_prev_year_mean'].isna()].groupby('Store')['Customers'].shift(0).rolling(180, min_periods = 0).mean()


In [228]:
#full_df['sales_per_customers_prev_year_mean'] = full_df['rolling_sales_prev_year_mean'] / full_df['rolling_customers_prev_year_mean']
#full_df['sales_per_customers_prev_year_med'] = full_df['rolling_sales_prev_year_med'] / full_df['rolling_customers_prev_year_med']
#full_df['sales_per_customers_prev_year_mean'].fillna(0 , inplace = True)
#full_df['sales_per_customers_prev_year_med'].fillna(0, inplace = True)

In [229]:
full_df.describe(include='all')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Id,IsStateHoliday,Year,Month,Day,Timestamp,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,CompetitionOpenSince,CompetitionDaysOpen,CompetitionIsOpen,Promo2Days,Promo2IsActive,AfterSchoolHoliday,BeforeSchoolHoliday,AfterIsStateHoliday,BeforeIsStateHoliday,AfterPromo,BeforePromo,rolling_sales_half_year_med,rolling_sales_prev_year_med,rolling_sales_prev_year2_med,rolling_sales_past_quarter_med
count,1058297.000,1058297.000,1058297,1017209.000,1017209.000,1058297.000,1058297.000,1058297,1058297.000,41088.000,1058297,1058297.000,1058297.000,1058297.000,1058297.000,1058297,1058297,1058297.000,1058297.000,533034,1058297,1058297.000,1058297.000,1058297.000,1058297.000,1058297.000,1058297.000,1058297.000,1058297.000,1058297.000,1058297.000,1004778.000,655783.000,662473.000,1004778.000
unique,nan,nan,990,nan,nan,nan,nan,4,nan,nan,2,nan,nan,nan,nan,4,3,nan,nan,3,163,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
top,nan,nan,2015-05-21 00:00:00,nan,nan,nan,nan,no,nan,nan,False,nan,nan,nan,nan,a,a,nan,nan,"Jan,Apr,Jul,Oct",2050-01-01 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
freq,nan,nan,1115,nan,nan,nan,nan,1027067,nan,nan,1027067,nan,nan,nan,nan,573755,557749,nan,nan,306898,338564,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
first,nan,nan,2013-01-01 00:00:00,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,2000-01-01 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
last,nan,nan,2015-09-17 00:00:00,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,2050-01-01 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
mean,558.331,3.998,NaN,5773.819,633.146,0.831,0.382,NaN,0.189,20544.500,NaN,2013.878,5.944,15.618,0.490,NaN,NaN,5416.793,0.504,NaN,NaN,1227.657,0.600,410.822,0.437,22.715,-22.791,38.297,-34.309,3.208,-3.105,5366.679,5342.583,5307.961,5408.205
std,321.846,1.998,NaN,3849.926,464.412,0.375,0.486,NaN,0.391,11861.228,NaN,0.795,3.298,8.785,0.289,NaN,NaN,7687.199,0.500,NaN,NaN,1487.435,0.490,590.108,0.496,21.938,23.099,31.808,32.794,5.827,3.323,2796.277,2862.824,2792.166,2846.913
min,1.000,1.000,NaN,0.000,0.000,0.000,0.000,NaN,0.000,1.000,NaN,2013.000,1.000,1.000,0.000,NaN,NaN,20.000,0.000,NaN,NaN,0.000,0.000,0.000,0.000,0.000,-194.000,0.000,-195.000,0.000,-16.000,0.000,0.000,0.000,0.000
25%,280.000,2.000,NaN,3727.000,405.000,1.000,0.000,NaN,0.000,10272.750,NaN,2013.000,3.000,8.000,0.240,NaN,NaN,710.000,0.000,NaN,NaN,0.000,0.000,0.000,0.000,2.000,-37.000,10.000,-57.000,0.000,-6.000,4921.500,4543.000,4671.000,4836.000


In [230]:
full_df.columns
full_df.head(3)
full_df.tail(3)
#full_df.info()

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Id', 'IsStateHoliday', 'Year',
       'Month', 'Day', 'Timestamp', 'StoreType', 'Assortment',
       'CompetitionDistance', 'Promo2', 'PromoInterval',
       'CompetitionOpenSince', 'CompetitionDaysOpen', 'CompetitionIsOpen',
       'Promo2Days', 'Promo2IsActive', 'AfterSchoolHoliday',
       'BeforeSchoolHoliday', 'AfterIsStateHoliday', 'BeforeIsStateHoliday',
       'AfterPromo', 'BeforePromo', 'rolling_sales_half_year_med',
       'rolling_sales_prev_year_med', 'rolling_sales_prev_year2_med',
       'rolling_sales_past_quarter_med'],
      dtype='object')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Id,IsStateHoliday,Year,Month,Day,Timestamp,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,CompetitionOpenSince,CompetitionDaysOpen,CompetitionIsOpen,Promo2Days,Promo2IsActive,AfterSchoolHoliday,BeforeSchoolHoliday,AfterIsStateHoliday,BeforeIsStateHoliday,AfterPromo,BeforePromo,rolling_sales_half_year_med,rolling_sales_prev_year_med,rolling_sales_prev_year2_med,rolling_sales_past_quarter_med
0,1,2,2013-01-01,0.000,0.000,0.000,0,a,1,nan,True,2013,1,1,0.000,c,a,1270.000,0,NaN,2008-09-01,1583,1,0,0,0,0,0,0,0,-6,nan,nan,nan,nan
1,2,2,2013-01-01,0.000,0.000,0.000,0,a,1,nan,True,2013,1,1,0.000,a,a,570.000,1,"Jan,Apr,Jul,Oct",2007-11-01,1888,1,1009,1,0,0,0,0,0,-6,nan,nan,nan,nan
2,3,2,2013-01-01,0.000,0.000,0.000,0,a,1,nan,True,2013,1,1,0.000,a,a,14130.000,1,"Jan,Apr,Jul,Oct",2006-12-01,2223,1,638,1,0,0,0,0,0,-6,nan,nan,nan,nan


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Id,IsStateHoliday,Year,Month,Day,Timestamp,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,CompetitionOpenSince,CompetitionDaysOpen,CompetitionIsOpen,Promo2Days,Promo2IsActive,AfterSchoolHoliday,BeforeSchoolHoliday,AfterIsStateHoliday,BeforeIsStateHoliday,AfterPromo,BeforePromo,rolling_sales_half_year_med,rolling_sales_prev_year_med,rolling_sales_prev_year2_med,rolling_sales_past_quarter_med
1058294,1113,4,2015-09-17,nan,nan,1.000,1,no,0,854.000,False,2015,9,17,1.000,a,c,9260.000,0,NaN,2050-01-01,0,0,0,0,20,0,115,0,0,0,8626.500,0.000,4437.500,8973.000
1058295,1114,4,2015-09-17,nan,nan,1.000,1,no,0,855.000,False,2015,9,17,1.000,a,c,870.000,0,NaN,2050-01-01,0,0,0,0,22,0,115,0,0,0,8637.000,0.000,4572.500,8973.000
1058296,1115,4,2015-09-17,nan,nan,1.000,1,no,0,856.000,False,2015,9,17,1.000,d,c,5350.000,1,"Mar,Jun,Sept,Dec",2050-01-01,0,0,1207,1,13,0,105,0,0,0,8641.000,0.000,5134.500,8999.000


In [231]:
cat_cols = ['StoreType', 'StateHoliday', 'Assortment']
formated_df = full_df

for col in cat_cols:
    formated_df= pd.merge(formated_df
                , pd.get_dummies(full_df[col], prefix=col)
                ,on = None, left_index=True, right_index=True)

#Keeping 'store' column
if 'Store' in cat_cols:
    cat_cols.remove('Store')

#Removed here to keep being able to visualize
#formated_df = formated_df.drop(cat_cols, axis = 1)

formated_df= formated_df.sort_values(['Timestamp', 'Store'])
formated_df.shape
formated_df.columns

(1058297, 46)

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Id', 'IsStateHoliday', 'Year',
       'Month', 'Day', 'Timestamp', 'StoreType', 'Assortment',
       'CompetitionDistance', 'Promo2', 'PromoInterval',
       'CompetitionOpenSince', 'CompetitionDaysOpen', 'CompetitionIsOpen',
       'Promo2Days', 'Promo2IsActive', 'AfterSchoolHoliday',
       'BeforeSchoolHoliday', 'AfterIsStateHoliday', 'BeforeIsStateHoliday',
       'AfterPromo', 'BeforePromo', 'rolling_sales_half_year_med',
       'rolling_sales_prev_year_med', 'rolling_sales_prev_year2_med',
       'rolling_sales_past_quarter_med', 'StoreType_a', 'StoreType_b',
       'StoreType_c', 'StoreType_d', 'StateHoliday_a', 'StateHoliday_b',
       'StateHoliday_c', 'StateHoliday_no', 'Assortment_a', 'Assortment_b',
       'Assortment_c'],
      dtype='object')

In [232]:
drop_cols = ["Customers", 'CompetitionOpenSince'
            #,'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c'
            #,'StateHoliday_no'
             ,'IsStateHoliday'
             ,'Promo2IsActive', 'index', 'PromoInterval', 'Id'
            ]
formated_df = formated_df.drop(drop_cols, axis = 1, errors ='ignore')

In [233]:
def find_null_cols(df):
    for column in df:
        if df[column].isnull().any():
            print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

find_null_cols(formated_df)

Sales has 41088 null values
rolling_sales_half_year_med has 53519 null values
rolling_sales_prev_year_med has 402514 null values
rolling_sales_prev_year2_med has 395824 null values
rolling_sales_past_quarter_med has 53519 null values


41088 null Sales corresponding to the Kaggle validation dataset

## Feather

In [234]:
formated_df[formated_df['Sales'].isna()].reset_index().to_feather('df_test')
formated_df[formated_df['Sales'].notna()].reset_index().to_feather('df_train')